<a href="https://colab.research.google.com/github/Ekliipce/Representation-and-Generative-Learning/blob/main/AutoEncoder_for_Image_Compression_(with_pytorch).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch

In [2]:
class AutoEncoder(torch.nn.Module):
  def __init__(self, batch_shape, channels, rows, cols, latent_dim):
    self.history = []
    self.batch_shape  = batch_shape
    self.channels = channels
    self.rows = rows
    self.cols = cols
    self.nb_input = self.rows * self.cols * self.channels

    self.encoder = torch.nn.Sequential([
        torch.nn.Flatten(),
        torch.nn.Linear(in_features=nb_inputs, out_features=128),
        torch.nn.ReLU(),
        torch.nn.Linear(in_features=128, out_features=64),
        torch.nn.ReLU(),
        torch.nn.Linear(in_features=64, out_features=32),
        torch.nn.ReLU(),
        torch.nn.Linear(in_features=32, out_features=self.latent_dim),
        torch.nn.ReLU()
    ])

    self.decoder = torch.nn.Sequential([
        torch.nn.Linear(in_features=self.latent_dim, out_features=32),
        torch.nn.ReLU(),
        torch.nn.Linear(in_features=32, out_features=64),
        torch.nn.ReLU(),
        torch.nn.Linear(in_features=64, out_features=124),
        torch.nn.ReLU(),
        torch.nn.Linear(in_features=128, out_features=nb_inputs),
        torch.nn.ReLU(),
        torch.nn.Unflatten(1, (self.channels, self.rows, self.cols))
    ])

  def forward(self, x):
    encode = self.encoder(x)
    decode = self.decoder(encode)
    return decode

  def train(self, x_train, y_train, epochs=10, lr=0.001):
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(self.parameters(), lr=lr)

    for epoch in epochs :
      output = self.forward(x_train)
      loss = criterion(output, y_train)

      optimizer.zero_grad()
      criterion.backward()
      optimizer.step()

      print(f"Epoch [{epoch+1}/epochs], Loss: {loss.item():.4f}")
